In [51]:
import numpy as np
import pandas as pd
from gekko import GEKKO
from pathlib import Path
import os

 
class CGE():
    
    def __init__(self, used_data_folder):

        work_dir = 'E:\\nauka\\GitHub\\cge_model_python'

        self.use = pd.read_csv(work_dir + '\\Data\\' + used_data_folder + '\\production.csv', index_col=0)
        self.xdem = pd.read_csv(work_dir + '\\Data\\' + used_data_folder + '\\consumption.csv', index_col=0)
        # self.xsup = pd.read_csv(work_dir + '\\Data\\' + used_data_folder + '\\economy_output.csv', index_col=0)
        self.intc = pd.read_csv(work_dir + '\\Data\\' + used_data_folder + '\\intermediate_consumption.csv', index_col=0)        
        self.enfac = pd.read_csv(work_dir + '\\Data\\' + used_data_folder + '\\endowment.csv', index_col=0)
        self.gdem = pd.read_csv(work_dir + '\\Data\\' + used_data_folder + '\\government_consumption.csv', index_col=0)
        #self.gsav = pd.read_csv(work_dir + '\\Data\\' + used_data_folder + '\\government_savings.csv', index_col=0)
        #self.tc = pd.read_csv(work_dir + '\\Data\\' + used_data_folder + '\\consumption_taxes.csv', index_col=0)
        self.ti = pd.read_csv(work_dir + '\\Data\\' + used_data_folder + '\\income_taxes.csv', index_col=0)
        
        self.factors = self.use.index
        self.sectors = self.use.columns
        self.households = self.xdem.columns
        
        self.m = GEKKO(remote=False)
        
        self.XS0 = {}
        self.VA0 = {}
        self.XDG0 = {}
        
        self.BETA = pd.DataFrame(index=self.factors, columns=self.sectors)
        self.AIJ = pd.DataFrame(index=self.sectors, columns=self.sectors)
        self.B = {}

        self.GAMMA = {}
        self.BBCES = pd.DataFrame(index=self.factors, columns=self.sectors)
        self.BCES = {}

        self.ALPHAG = {}

        self.ENDW0 = pd.DataFrame(index=self.factors, columns=self.households)
        self.XD0 = pd.DataFrame(index=self.sectors, columns=self.households)
        self.trans = pd.DataFrame(0, index=self.households, columns=self.households)
        self.ALPHA = pd.DataFrame(index=self.sectors, columns=self.households)
        self.tr_in = {}
        self.tr_out = {}
        self.INC0 = {}
        self.W0 = {}
        self.A = {}
        self.ITAX = {}
        self.TAXREV0 = {}
    
        self.FTAX = pd.DataFrame(index=self.sectors, columns=self.factors)
        self.factor_shock = {}
        for fac in self.factors:
            self.factor_shock[fac] = 0
            for sec in self.sectors:
                self.FTAX[fac][sec] = 0


    def m_sum(self, array):
        return np.sum(np.nan_to_num(array)) 
    def m_prod(self, array):
        return np.prod(np.nan_to_num(array))
    def m_nan(self, value):
        return np.nan_to_num(value)


    def parameters_extended(self):

        # Recalibration of output including intermediate use (Leontief)
        for sec in self.sectors:
            self.XS0[sec] = self.m_nan(
                sum(self.intc[sec]) + sum(self.use[sec])
            )
            self.VA0[sec] = self.m_nan(
                sum(self.use[sec])
            )
            for fac in self.factors:
                self.BETA[sec][fac] = self.m_nan(
                    self.use[sec][fac] / self.VA0[sec]
                )
            self.B[sec] = self.m_nan(
                self.VA0[sec] / self.m_prod([
                    self.use[sec][fac]**self.BETA[sec][fac] for fac in self.factors
                ])
            )
            for sect in self.sectors:
                self.AIJ[sec][sect] = self.m_nan(
                    self.intc[sec][sect] / self.XS0[sec]
                )
        # CES production function calibration
        for sec in self.sectors:
            self.GAMMA[sec] = 0.5
            for fac in self.factors:
                self.BBCES[sec][fac] = self.m_nan(
                    (self.use[sec][fac]**(1/self.GAMMA[sec])) / self.m_sum([
                        self.use[sec][fact]**(1/self.GAMMA[sec])
                        for fact in self.factors
                    ])**self.GAMMA[sec]
                )
            self.BCES[sec] = self.m_nan(
                self.VA0[sec] / self.m_sum([
                    self.BBCES[sec][fac]**(1/self.GAMMA[sec]) 
                    * self.use[sec][fac]**((self.GAMMA[sec] - 1)/self.GAMMA[sec])
                    for fac in self.factors
                ])**(self.GAMMA[sec]/(self.GAMMA[sec] - 1))
            )
        # Calibration of the government
        for sec in self.sectors:
            self.XDG0[sec] = self.m_nan(
                self.gdem['GOV'][sec]
            )
        # Parameters of govt C-D function
        self.WG0 = self.m_sum([
            self.XDG0[sec] for sec in self.sectors
        ])
        for sec in self.sectors:
            self.ALPHAG[sec] = self.m_nan(
                self.XDG0[sec] / self.m_sum([
                    self.XDG0[sect] 
                    for sect in self.sectors
                ])
            )
        self.AG = self.m_nan(
            self.WG0 / self.m_prod([
                self.XDG0[sec]**self.ALPHAG[sec]
                for sec in self.sectors
            ])
        )
        # Households parameters
        for hou in self.households:
            for fac in self.factors:
                self.ENDW0[hou][fac] = self.m_nan(
                    self.enfac[fac][hou] * (1 - self.factor_shock[fac])
                )
            self.INC0[hou] = self.m_sum([
                self.ENDW0[hou][fac]
                for fac in self.factors
            ])
            for sec in self.sectors:
                self.XD0[hou][sec] = self.m_nan(
                    self.xdem[hou][sec]
                )
            # benchmark utility
            self.W0[hou] = self.m_sum([
                self.XD0[hou][sec]
                for sec in self.sectors
            ])
            for sec in self.sectors:
                self.ALPHA[hou][sec] = self.m_nan(
                    self.XD0[hou][sec] / self.W0[hou]
                )
            self.A[hou] = self.m_nan(
                self.W0[hou] / self.m_prod([
                    self.XD0[hou][sec]**self.ALPHA[hou][sec]
                    for sec in self.sectors
                ])
            )
            self.TAXREV0[hou] = self.m_sum(
                self.ti[hou]
            )
            self.ITAX[hou] = self.m_nan(
                self.TAXREV0[hou] / self.INC0[hou]
            )
            # Transfers
            self.tr_in[hou] = self.m_nan(
                self.trans[hou].sum()
            )
            self.tr_out[hou] = self.m_nan(
                self.trans.loc[hou].sum()
            )
        
        # Variables
        self.VA = {}
        self.PVA = {}
        self.XS = {}
        self.P = {}
        for sec in self.sectors:
            self.VA[sec] = self.m.Var(lb=0, value=self.VA0[sec])
            self.PVA[sec] = self.m.Var(lb=0, value=1)
            self.XS[sec] = self.m.Var(lb=0, value=self.XS0[sec])
            self.P[sec] = self.m.Var()
            
            self.ALPHAG[sec] = self.ALPHAG[sec]
            self.BCES[sec]
            self.GAMMA[sec]
            self.VA[sec]
            self.B[sec]
            for sect in self.sectors:
                self.AIJ[sec][sect]
        self.INC = {}
        self.W = {}
        self.PW = {}
        for hou in self.households:
            self.INC[hou] = self.m.Var()
            self.W[hou] = self.m.Var()
            self.PW[hou] = self.m.Var()
            
            self.ITAX[hou] = self.ITAX[hou]
            for sec in self.sectors:
                self.ALPHA[hou][sec]
            for fac in self.factors:
                self.ENDW0[hou][fac]
        self.PFAC = {}
        for fac in self.factors:
            self.PFAC[fac] = self.m.Var()
            
            for sec in self.sectors:
                self.FTAX[fac][sec] = self.FTAX[fac][sec]
                self.BBCES[sec][fac] = self.BBCES[sec][fac]
        self.PWG = self.m.Var()
        self.WG = self.m.Var()
        self.INCG = self.m.Var()
        self.GSAV = self.m.Var()
        
        self.AG = self.AG


    def equilibrium_extended(self):

        self.parameters_extended()

        self.equations = {
            'PRF_WG': (
                self.m_prod([
                    (self.P[sec] / self.ALPHAG[sec])**(self.ALPHAG[sec]) 
                    for sec in self.sectors
                ]) / self.AG
                == self.PWG
            ),
            'MKT_WG': (
                self.PWG * self.WG
                == self.INCG - self.GSAV
            ),
            'I_INCG': (
                self.m_sum([
                    self.INC[hou] * self.ITAX[hou]
                    for hou in self.households
                ])
                + self.m_sum([
                    self.m_sum([
                        self.FTAX[fac][sec] * self.PFAC[fac]
                        * self.BBCES[sec][fac] * self.BCES[sec]**(self.GAMMA[sec] - 1)
                        * self.VA[sec] * self.PVA[sec]**self.GAMMA[sec]
                        / (self.PFAC[fac] * (1 + self.FTAX[fac][sec]))**(self.GAMMA[sec])
                        for sec in self.sectors
                    ])
                    for fac in self.factors
                ])
                == self.INCG
            )
        }
        for sec in self.sectors:
            self.equations[f'PRF_XS({sec})'] = (
                (1 - self.m_sum([
                    self.AIJ[sec][sect]
                    for sect in self.sectors
                ]))*self.PVA[sec]
                + self.m_sum([
                    self.AIJ[sec][sect] * self.P[sect]
                    for sect in self.sectors
                ])
                == self.P[sec]
            )
            self.equations[f'PRF_VA({sec})'] = (
                self.m_sum([
                    self.BBCES[sec][fac] * (self.PFAC[fac] * (1 + self.FTAX[fac][sec]))**(1 - self.GAMMA[sec])
                    for fac in self.factors
                ])
                == (self.B[sec] * self.PVA[sec])**(1 - self.GAMMA[sec])
            )
            self.equations[f'MKT_XS({sec})'] = (
                self.m_sum([
                    self.ALPHA[hou][sec] * self.W[hou] * self.PW[hou] / self.P[sec]
                    for hou in self.households
                ])
                + self.ALPHAG[sec] * self.WG * self.PWG / self.P[sec]
                + self.m_sum([
                    self.AIJ[sec][sect] * self.XS[sect]
                    for sect in self.sectors
                ])
                == self.XS[sec]
            )
            self.equations[f'MKT_VA({sec})'] = (
                (1 - self.m_sum([
                    self.AIJ[sec][sect] * self.XS[sec]
                    for sect in self.sectors
                ]))
                == self.VA[sec]
            )
        for hou in self.households:
            self.equations[f'PRF_W({hou})'] = (
                self.m_prod([
                    (self.P[sec] * self.ALPHA[hou][sec])**self.ALPHA[hou][sec]
                    for sec in self.sectors
                ])
                / self.A[hou]
                == self.PW[hou]
            )
            self.equations[f'MKT_W({hou})'] = (
                self.PW[hou] * self.W[hou]
                == self.INC[hou] * (1 - self.ITAX[hou])
            )
            self.equations[f'I_INC({hou})'] = (
                self.m_sum([
                    self.PFAC[fac] * self.ENDW0[hou][fac]
                    for fac in self.factors
                ])
                == self.INC[hou]
            )
        for fac in self.factors:
            self.equations[f'MKT_FAC({fac})'] = (
                self.m_sum([
                    self.BBCES[sec][fac] * self.BCES[sec]**(self.GAMMA[sec] - 1)
                    * self.VA[sec] * self.PVA[sec]**self.GAMMA[sec] 
                    / (self.PFAC[fac] * (1 + self.FTAX[fac][sec]))**self.GAMMA[sec]
                    for sec in self.sectors
                ])
                == self.m_sum([
                    self.ENDW0[hou][fac]
                    for hou in self.households
                ])
            )
        
        self.m.Equations(list(self.equations.values()))
        self.m.options.SOLVER=1
        self.m.solve()

In [52]:
test = CGE('Test')
test.equilibrium_extended()

 ----------------------------------------------------------------
 APMonitor, Version 0.9.2
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  25
   Intermediates:  0
   Connections  :  0
   Equations    :  24
   Residuals    :  24
 
 Number of state variables:    25
 Number of total equations: -  24
 Number of slack variables: -  0
 ---------------------------------------
 Degrees of freedom       :    1
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
 
 Iter    Objective  Convergence
    0  9.92098E-16  1.88593E+00
    1         +Inf  1.88593E+00
    2         +Inf  1.88593E+00
    3         +Inf  1.88593E+00
    4         +Inf  1.88593E+00
    5         +Inf  1.88593E+00
    6         +Inf  1.88593E+00
    7         +I

Exception: @error: Solution Not Found


In [50]:
test.TAXREV0

{'HOUSE1': i513}

In [16]:
x = np.inf
x

inf

In [17]:
np.isnan(x)

False

In [ ]:
self.m.Const